In [1]:
# import libraries
import numpy as np
import pandas as pd

In [2]:
# import data
dir = 'C:\\Users\\Lenovo\\PycharmProjects\\Kaggle\\Project2_Expedia\\'
chunker = pd.read_csv(dir + 'train.csv', nrows=100000)
train = pd.DataFrame(columns=[])
train_set = None
test_set = pd.read_csv(dir + 'test.csv')
dest_set  = pd.read_csv(dir + 'destinations.csv')

In [10]:
train_set.head().transpose()

,0,1,2,3,4
date_time,2014-08-11 07:46:59,2014-08-11 08:22:12,2014-08-11 08:24:33,2014-08-09 18:05:16,2014-08-09 18:08:18
site_name,2,2,2,2,2
posa_continent,3,3,3,3,3
user_location_country,66,66,66,66,66
user_location_region,348,348,348,442,442
user_location_city,48862,48862,48862,35390,35390
orig_destination_distance,2234.26,2234.26,2234.26,913.193,913.626
user_id,12,12,12,93,93
is_mobile,0,0,0,0,0
is_package,1,1,0,0,0


In [11]:
test_set.head().transpose()

,0,1,2,3,4
id,0,1,2,3,4
date_time,2015-09-03 17:09:54,2015-09-24 17:38:35,2015-06-07 15:53:02,2015-09-14 14:49:10,2015-07-17 09:32:04
site_name,2,2,2,2,2
posa_continent,3,3,3,3,3
user_location_country,66,66,66,66,66
user_location_region,174,174,142,258,467
user_location_city,37449,37449,17440,34156,36345
orig_destination_distance,5539.06,5873.29,3975.98,1508.6,66.7913
user_id,1,1,20,28,50
is_mobile,1,1,0,0,0


## Let's explore both sets:
1. Common columns
2. Dimensions
2. dtypes
3. unique values (for categorization)
4. Nulls
5. 
5.

In [105]:
# COMMON COLUMNS
setA = pd.DataFrame({'Train': train_set.columns})
setB = pd.DataFrame({'Train': test_set.columns, 'Test': ['X' for x in range(test_set.shape[1])]})
pd.merge(setA, setB, how='outer', left_on='Train', right_on='Train')

,Train,Test
0,date_time,X
1,site_name,X
2,posa_continent,X
3,user_location_country,X
4,user_location_region,X
5,user_location_city,X
6,orig_destination_distance,X
7,user_id,X
8,is_mobile,X
9,is_package,X


In [126]:
# Dimensions
train_set.shape, test_set.shape

((100000, 24), (2528243, 22))

In [42]:
# DATA TYPES
train_set.dtypes

date_time                     object
site_name                      int64
posa_continent                 int64
user_location_country          int64
user_location_region           int64
user_location_city             int64
orig_destination_distance    float64
user_id                        int64
is_mobile                      int64
is_package                     int64
channel                        int64
srch_ci                       object
srch_co                       object
srch_adults_cnt                int64
srch_children_cnt              int64
srch_rm_cnt                    int64
srch_destination_id            int64
srch_destination_type_id       int64
is_booking                     int64
cnt                            int64
hotel_continent                int64
hotel_country                  int64
hotel_market                   int64
hotel_cluster                  int64
dtype: object

In [127]:
# % OF NULL VALUES
setA = (train_set.isnull().sum() / train_set.shape[0]) * 100
setB = (test_set.isnull().sum() / test_set.shape[0]) * 100
pd.concat([setA, setB], join='outer', axis=1, keys=('Train', 'Test'))

,Train,Test
channel,0.000,0.000000
cnt,0.000,NaN
date_time,0.000,0.000000
hotel_cluster,0.000,NaN
hotel_continent,0.000,0.000000
hotel_country,0.000,0.000000
hotel_market,0.000,0.000000
id,NaN,0.000000
is_booking,0.000,NaN
is_mobile,0.000,0.000000


In [131]:
# UNIQUE VALUES IN EACH COLUMN
train_set.apply(lambda x: x.value_counts().count())
#setB = test_set.apply(lambda x: x.value_counts().count())
#pd.concat([setA, setB], axis=1, keys=('Train', 'Test'))

date_time                    99840
site_name                       38
posa_continent                   5
user_location_country          123
user_location_region           510
user_location_city            3751
orig_destination_distance    43021
user_id                       3478
is_mobile                        2
is_package                       2
channel                         11
srch_ci                       1029
srch_co                       1034
srch_adults_cnt                 10
srch_children_cnt               10
srch_rm_cnt                      9
srch_destination_id           5296
srch_destination_type_id         8
is_booking                       2
cnt                             30
hotel_continent                  6
hotel_country                  152
hotel_market                  1531
hotel_cluster                  100
dtype: int64

In [123]:
lista = []
for col in range(len(test_set.columns)):
    lista.append(test_set.iloc[:, col].value_counts().count())
pd.Series(lista, index=test_set.columns)

id                           2528243
date_time                    2399366
site_name                         53
posa_continent                     5
user_location_country            236
user_location_region             988
user_location_city             36722
orig_destination_distance    1344826
user_id                      1181577
is_mobile                          2
is_package                         2
channel                           11
srch_ci                          778
srch_co                          772
srch_adults_cnt                   10
srch_children_cnt                 10
srch_rm_cnt                        9
srch_destination_id            40718
srch_destination_type_id           8
hotel_continent                    7
hotel_country                    206
hotel_market                    2115
dtype: int64

## Prepare Datasets for analysis:
1. convert dates to dates format
2. 

In [133]:
train_set['date_time'] = pd.to_datetime(train_set.date_time)

In [138]:
train_set.date_time

0       2014-08-11 07:46:59
1       2014-08-11 08:22:12
2       2014-08-11 08:24:33
3       2014-08-09 18:05:16
4       2014-08-09 18:08:18
5       2014-08-09 18:13:12
6       2014-07-16 09:42:23
7       2014-07-16 09:45:48
8       2014-07-16 09:52:11
9       2014-07-16 09:55:24
10      2014-07-16 10:00:06
11      2014-07-16 10:02:58
12      2014-01-17 06:24:56
13      2014-01-18 14:33:31
14      2014-01-21 06:39:08
15      2014-01-21 06:40:18
16      2014-01-22 06:10:02
17      2014-01-24 11:52:04
18      2014-01-24 17:26:24
19      2014-02-27 17:44:23
20      2014-02-27 18:01:32
21      2014-12-30 09:50:51
22      2014-12-30 09:55:43
23      2014-12-30 09:56:26
24      2014-10-22 12:20:56
25      2014-11-10 12:38:02
26      2013-06-15 15:10:49
27      2013-06-15 15:38:05
28      2014-11-22 14:32:46
29      2014-11-22 14:37:05
                ...        
99970   2013-03-13 09:08:59
99971   2013-03-13 09:09:34
99972   2013-03-13 09:12:23
99973   2013-03-13 09:12:40
99974   2013-03-13 0